<center> <h1> UGCA Assignment 3 </h1> </center>
<center> <h3> Matt Viteri, Colin Chu, Darasimi Oluwaniyi, Vikram Gupta, & Yeggi Lee </h3> </center>

<b> High level description: </b> The objective of this group assignment is to create the building blocks of a crowdsourced recommendation system. This recommendation system should accept user inputs about desired attributes of a product and come up with 3 recommendations. In this case, we used beer recommendations from the following website: https://www.beeradvocate.com/beer/top-rated/

In [1]:
import numpy as np
import pandas as pd
import re
import textblob
import nltk
import matplotlib.pyplot as plt
import random

from sklearn import manifold
from configparser import ConfigParser
from IPython.display import display
from collections import Counter
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

stop_words = set(stopwords.words('english'))

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/abhivik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### TO KEEP IN MIND: 

Your submission should include the following: 
1. Names of all team members inside the document (only one submission per team)
2. All scripts 
3. The average sentiment and similarity scores for the three products you recommended in task E.
4. Your analyses and answer to task F. Make sure you show the ratings, similarity scores and sentiments for the products you recommend in tasks E and F. 

#### Guidelines
1.    For similarity analysis, use spacy (instead of simple cosine similarity). The script should accept as input a file with the product attributes, and calculate similarity scores (between 0 and 1) between these attributes and each review. That is, the output file should have 3 columns – product_name, product_review and similarity_score. 
2.    The same beer may appear on multiple rows (in the 300 rows with highest similarity scores) since there are multiple reviews for each beer. Reduce the number of rows by averaging the user_rating of each beer (can do it in Excel). After this operation, each beer will be present only in one row. The user_rating column will now have average ratings for each beer. 
3.    For task F, you do the same averaging, but with ALL rows of data (that is, as many rows as there are reviews).   

### Task A
Extract about 6k reviews of beers from Beeradvocate.com. It is a single-page listing of 250 top-rated beers (avoids the pagination feature, which you need in cases where listings go on for many pages). Scrape 25 reviews and well as the ratings provided by users for each beer. The way beeradvocate.com organizes reviews is that it provides 25 reviews per page, which means you can avoid the pagination part for the reviews as well. The CSV output file should have 3 columns: product_name, product_review, and user_rating. 
- The scrapper is in the file, "scraper.py" and the output is in "output.csv"
- Approximately ____k reviews were scraped

In [2]:
orig_df = pd.read_csv('output.csv')
orig_df.head(5)

,beer,look,smell,taste,feel,overall,comment
0,Kentucky Brunch Brand Stout,4.75,4.75,4.75,4.75,5.00,"2016 Silver Wax. Aroma has whiskey, maple, tof..."
1,Kentucky Brunch Brand Stout,4.50,4.75,4.75,4.75,4.75,The beer pours Pitch Black with a frothy tan h...
2,Kentucky Brunch Brand Stout,4.75,4.75,4.50,5.00,4.75,Probably the smoothest beer I have ever had. S...
3,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,"Dark black, very thick, a little bit of tan he..."
4,Kentucky Brunch Brand Stout,4.50,5.00,5.00,5.00,5.00,Poured black as ink with thin ruby edges at 58...


### Task B
Assume that a customer, who will be using this recommendation system, has specified 3 attributes in a beer. One website describes multiple attributes of beer: https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/
- <b> Aggressive: </b> Boldly assertive aroma and/or taste
- <b>Balanced:</b> Malt and hops in similar proportions; equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish
- <b>Complex:</b> Multidimensional; many flavors and sensations on the palate
- <b>Crisp: </b>Highly carbonated; effervescent
- <b>Fruity:</b> Flavors reminiscent of various fruits or <b>Hoppy: </b> Herbal, earthy, spicy, or citric aromas and flavors of hops or <b>Malty</b>: Grainy, caramel-like; can be sweet or dry
- <b>Robust:</b> Rich and full-bodied

Choose three attributes from the above list. If you choose another product (like electric guitars), choose a different set – e.g., action, neck, pickup (for guitars). It is a good idea to perform a word frequency analysis of the data you have collected to make sure that the above attributes are actually mentioned.

In [3]:
top50_beers = ['Kentucky Brunch Brand Stout', 'Marshmallow Handjee', "Hunahpu's Imperial Stout - Double Barrel Aged", 'Barrel-Aged Abraxas', 'Heady Topper', 'King Julius', 'Pliny The Younger', "Mornin' Delight", 'Very Hazy', 'King JJJuliusss', 'Fundamental Observation', 'SR-71', 'CBS (Canadian Breakfast Stout)', 'Juice Machine', 'Pliny The Elder', "Fou' Foune", 'Assassin', 'Very Green', 'Julius', 'Double Sunshine', 'Zenne Y Frontera', 'Abner', 'Bourbon County Brand Stout', 'Trappist Westvleteren 12 (XII)', 'Dinner', 'KBS (Kentucky Breakfast Stout)', 'JJJuliusss', 'Barrel Aged Imperial German Chocolate Cupcake Stout', 'Parabola', 'Duck Duck Gooze', 'Speedway Stout - Vietnamese Coffee - Bourbon-Barrel Aged', 'Morning Wood', 'Lou Pepe - Kriek', 'Zombie Dust', 'Westly', 'Bourbon County Brand Vanilla Stout', 'Doubleganger', 'Fuzzy', 'Double Citra®', 'Ann', "Hunahpu's Imperial Stout", 'King Sue', 'Atrial Rubicite', 'Headroom', 'Double Galaxy', 'Supplication', 'Cable Car', 'Citra', 'Framboise Du Fermier', 'Focal Banger', 'Double Dry Hopped Congress Street', 'Very GGGreennn', 'Green', 'Lou Pepe - Framboise', 'Aaron', 'Sip Of Sunshine', 'Black Tuesday', 'Swish', 'BBADL (Bourbon Barrel Aged Dark Lord Imperial Stout)', 'Haze']
top_valid_beers = (orig_df[orig_df['beer'].isin(top50_beers)]
                   .groupby('beer')['beer']
                   .count()
                   .where(lambda v: v > 100)
                   .dropna()
                   .sort_values(ascending=False)
                   .index
                   .tolist())

df = orig_df[orig_df['beer'].isin(top_valid_beers)].groupby('beer').head(200)

In [4]:
same_map = {
    'aggresive': ['bold', 'assertive'],
    'balanced': ['balance', 'multidimensional'],
    'complex': ['complexing'],
    'crisp': ['crispy', 'carbonated', 'carbonation'],
    'fruity': ['fruit', 'tart', 'apple', 'fig', 'orange', 'apples', 'oranges', 'raspberry', 'raspberries', 
               'cherry', 'grapefruit', 'tropical'],
    'hoppy': ['herbal', 'earthy', 'spicy', 'citric', 'hops', 'hop', 'citrus'],
    'malty': ['malt', 'grainy', 'caramel', 'maple'],
    'robust': ['rich', 'full-bodied']
}

same_inv_map = {word: root for root, words in same_map.items() for word in words}

def splitter(data):
    words = re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", data.lower()).split()
    return [textblob.Word(word).lemmatize() for word in words]

def convert_same(text):
    return [(same_inv_map[word] if word in same_inv_map else word) for word in splitter(text)]

def wordCount(data):
    filtered = [w for w in convert_same(' '.join(data['comment'])) if w not in stop_words]
    return Counter(filtered).most_common(25)

df['split_comments'] = df['comment'].map(convert_same)

wordCount(df)

[('fruity', 15966),
 ('beer', 10937),
 ('hoppy', 8637),
 ('taste', 5713),
 ('head', 5536),
 ('wa', 5278),
 ('malty', 5274),
 ('flavor', 4908),
 ('like', 4222),
 ('one', 4159),
 ('crisp', 3644),
 ('smell', 3510),
 ('nice', 3433),
 ('aroma', 3360),
 ('vanilla', 3308),
 ('dark', 3287),
 ('sweet', 3284),
 ('good', 3205),
 ('note', 3155),
 ('well', 3081),
 ('chocolate', 3051),
 ('finish', 3019),
 ('bourbon', 2859),
 ('nose', 2810),
 ('bottle', 2797)]

#### Results
The most common attributes out of the list above are: 
- Fruity
- Hoppy
- Malty 
- Crisp
- Balanced
- Robust
- Complex

In [5]:
attributes = ['crisp', 'balanced', 'fruity']

### Task C 
Perform a similarity analysis with the 3-attribute set and the reviews. See guidelines below on how to do this. From the output file, choose 300 reviews that have the highest similarity scores with the attribute set you have chosen. 

Run the following line in your terminal AFTER you install spacy
<br />
`python -m spacy download en_core_web_lg`

In [6]:
import spacy

nlp = spacy.load("en_core_web_lg")
STOPWORDS = nlp.Defaults.stop_words

In [7]:
attstr = ' '.join(attributes)

In [8]:
# ignore stopwords, punctuation and pronouns
def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        result.append(token.lemma_)
    return ' '.join(result)

In [9]:
def calculate_similarity(text):
    base = nlp(attstr)
    compare = nlp(process_text(text))
    return base.similarity(compare)

In [10]:
sims = []

for i, r in df.iterrows():
    s = calculate_similarity(r.comment)
    d = {
        'product_name': r.beer,
        'product_review': r.comment,
        'user_rating': r.overall,
        'similarity_score': s
    }
    sims.append(d)

In [11]:
sims_df = pd.DataFrame(sims)
sims_df.sort_values(by=['similarity_score'], ascending=False, inplace=True)
sims_df[:300]

,product_name,product_review,user_rating,similarity_score
4305,Duck Duck Gooze,Aroma - Prominent notes of acetic/citric acid ...,4.75,0.843623
4904,Zombie Dust,"Color is a perfect golden ale color, pleasant ...",4.75,0.839829
2102,Cable Car,Hazy golden colour with a large fluffy head. N...,4.50,0.838208
3972,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...,4.50,0.837662
7133,Pliny The Elder,Pours a clear vibrant marigold colour. Citrusy...,4.75,0.837084
...,...,...,...,...
3655,Zombie Dust,"This is a really good, really drinkable, and r...",4.50,0.793154
185,Green,Wonderful NE style IPA pours a beautiful hazy ...,4.75,0.793009
1630,Focal Banger,"Cloudy, gold, with two fingers of thick white ...",4.50,0.792863
2646,King Sue,Pours a bright and sunny yellow with soft oran...,4.50,0.792699


In [12]:
sims_df.to_csv('similarity_score.csv', index=None)

### Task D. 
Perform sentiment analysis on these 300 reviews and sort them (high to low) by the sentiment scores.

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']

In [14]:
df_sims_sentiment = pd.read_csv('similarity_score.csv')
df_sims_sentiment['Sentiment Score'] = df_sims_sentiment['product_review'].map(lambda x: sentiment_analyzer_scores(x))

In [15]:
df_sims_sentiment.sort_values(by=['Sentiment Score'], ascending=False, inplace=True)
df_sims_sentiment.to_csv('similarity_score&sentiment_score.csv', index=None)

In [16]:
df_sentiment_300 = pd.read_csv('similarity_score.csv')
df_sentiment_300 = df_sentiment_300[:300]
df_sentiment_300.head()

,product_name,product_review,user_rating,similarity_score
0,Duck Duck Gooze,Aroma - Prominent notes of acetic/citric acid ...,4.75,0.843623
1,Zombie Dust,"Color is a perfect golden ale color, pleasant ...",4.75,0.839829
2,Cable Car,Hazy golden colour with a large fluffy head. N...,4.50,0.838208
3,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...,4.50,0.837662
4,Pliny The Elder,Pours a clear vibrant marigold colour. Citrusy...,4.75,0.837084


In [17]:
df_sentiment_300['Sentiment Score'] = df_sentiment_300['product_review'].map(lambda x: sentiment_analyzer_scores(x))
df_sentiment_300.sort_values(by=['Sentiment Score'], ascending=False, inplace=True)
df_sentiment_300.head()

,product_name,product_review,user_rating,similarity_score,Sentiment Score
262,Lou Pepe - Framboise,2008 vintage purchased in Brussels from Bier T...,4.50,0.794714,0.9984
214,Abner,brew was poured from a refillable 750mL Hill F...,4.50,0.797283,0.9980
299,Atrial Rubicite,"500ml bottle, blend 3 (November, 2013), poured...",4.50,0.792659,0.9970
63,Green,"16 ounce can into tulip glass, canned on 3/8/2...",4.50,0.813388,0.9970
137,Fou' Foune,"750ml, bottled august 2012 A- Ligh orange col...",4.25,0.802747,0.9964


In [18]:
df_sentiment_300.to_csv('taskD.csv', index=None)

### Task E. 

Based on tasks C and D, <b> recommend 3 beers </b> to the customer. Note that in task D, multiple reviews may refer to the same beer. Show the average similarity and sentiment scores for each of the 3 recommended products. 

In [19]:
df_recommend = pd.read_csv('taskD.csv')

In [20]:
df_top3 = df_recommend.groupby(['product_name'])['Sentiment Score','similarity_score','user_rating'].mean().sort_values(by='Sentiment Score',ascending=False).head(3)
df_top3

,Sentiment Score,similarity_score,user_rating
product_name,,,
Lou Pepe - Kriek,0.966750,0.803238,4.625000
Cable Car,0.959856,0.806064,4.555556
Fundamental Observation,0.936600,0.804914,5.000000


In [21]:
print("The top 3 beers according to the similarity scores are",', '.join(df_top3.index))

The top 3 beers according to the similarity scores are Lou Pepe - Kriek, Cable Car, Fundamental Observation


### Task F. 
How would your recommendations differ if you ignored the similarity and sentiment scores and simply chose the 3 highest rated beers from your entire dataset? To do this, you need to calculate the average rating for each beer mentioned (remember each beer in your data has multiple reviews, and you need to get the average rating for each beer). Would these three beers meet the requirements of the user looking for recommendations? Why or why not? Justify your answer. Use the user_rating data, similarity and sentiment scores to answer this question. 

In [22]:
df_top3rating = df.groupby(['beer'])['overall'].mean().sort_values(ascending=False).head(3)
df_top3rating

beer
Kentucky Brunch Brand Stout    4.851449
Very Hazy                      4.774648
King Julius                    4.737730
Name: overall, dtype: float64

In [23]:
print("The top 3 beers according to the user ratings are",', '.join(df_top3rating.index))

The top 3 beers according to the user ratings are Kentucky Brunch Brand Stout, Very Hazy, King Julius


In [24]:
user_rating = list(df_top3rating.index)

### Top beer recommendation based on sentiment analysis & similarity score (averaging scores from 300 reviews with attribute mentions

In [25]:
df_top3

,Sentiment Score,similarity_score,user_rating
product_name,,,
Lou Pepe - Kriek,0.966750,0.803238,4.625000
Cable Car,0.959856,0.806064,4.555556
Fundamental Observation,0.936600,0.804914,5.000000


### Top beer solely based on user ratings (Data gotten from averaging overall reviews)

In [26]:
df_top_rating_3 = df_sims_sentiment[df_sims_sentiment['product_name'].isin(user_rating)]
df_top_rating_3.groupby(['product_name'])['Sentiment Score','similarity_score','user_rating'].mean().sort_values(by='user_rating',ascending=False)

,Sentiment Score,similarity_score,user_rating
product_name,,,
Kentucky Brunch Brand Stout,0.840588,0.638554,4.851449
Very Hazy,0.784270,0.688320,4.774648
King Julius,0.803991,0.678866,4.737730


#### The recommendation system is based on these following attributes: crisp, balanced, and fruity which we are assuming the user wants and looks for in a beer. On the other hand, the top three beers above  is based solely on user rating and is not necessarily catered to our user. This can be shown by comparing the similarity scores of the beers that our recommendation system gave and the ones above. The similarity scores of our recommendation system are roughly around 0.80 while the ones shown above are lower by at least 0.10. This is further proved by the sentiment scores. The ones from our recommendation system almost reach a sentiment score of 1.0 which means that the user really enjoys these beers as opposed to the sentiment scores of the top three which are roughly around 0.80. 